# TP1 - Grupo 14

André Lucena Ribas Ferreira - A94956

Paulo André Alegre Pinto - A97391

1. Para cada Timeslot, cada Líder tem de participar do seu próprio Projeto.

Isto pode ser traduzido da seguinte maneira:

1.1 Para cada Timeslot, cada Colaborador participa de um Projeto se o seu Líder também participar

$$ tex $$

In [ ]:
#Cada Líder tem de participar do seu projeto OU SEJA
    #Cada Colaborador participa apenas de um projeto se o seu líder participar
for t in xDict:
    for s in xDict[t]:
        for p in xDict[t][s]:
            for c in xDict[t][s][p]:
                solver.Add(xDict[t][s][p][c] <= xDict[t][s][p][projDict[p]['lider']])

2. Para cada Timeslot, cada Colaborador participa apenas de um Projeto.
$$ tex $$

In [23]:
from ortools.linear_solver import pywraplp

T, S, P, C = 4, 2, 3, 5    #período de 8 horas; 2 salas

projetos = {}
projetos[0] = {'colaboradores':[0,1,2,4], 'lider':4, 'min':1}
projetos[1] = {'colaboradores':[1,2,3], 'lider':1, 'min':2}
projetos[2] = {'colaboradores':[0,2,3,4], 'lider':3, 'min':2}

matrix = {}
matrix[0] = [1,0,1,1]
matrix[1] = [1,1,1,1]
matrix[2] = [1,1,0,1]
matrix[3] = [1,1,1,0]
matrix[4] = [1,1,1,0]

def scheduleStartUp(S, T, P, C, projDict, dispMatrix):
    xDict = {}
    result = dict()
    solver = pywraplp.Solver.CreateSolver('SCIP')
    #dicionário com T*P*C*S variáveis -> xDict[t][p][c][s] == 1 sse c estará presente na reunião do projeto p no timeslot t na sala s.
    for t in range(T):
        xDict[t] = {}
        for s in range(S):
            xDict[t][s] = {}
            for p in range(P):
                if dispMatrix[projDict[p]['lider']][t] == 1:
                    xDict[t][s][p] = {}
                    #Cada Colaborador apenas pode participar dos seus projetos -> Obrigatório pela definição dos dicionários
                    for c in projDict[p]['colaboradores']:
                        #Cada Colaborador tem de ter horário -> Obrigatório pela definição dos dicionários
                        if dispMatrix[c][t] == 1:
                            xDict[t][s][p][c] = solver.BoolVar('x[%i][%i][%i}[%i]' % (t,s,p,c))
                
    #Para cada Timeslot,
    for t in xDict:
        #Cada Sala apenas pode ter um projeto
        for s in xDict[t]:
            solver.Add(sum([xDict[t][s][p][projDict[p]['lider']] for p in xDict[t][s]]) <= 1)
            
        #Cada Colaborador possível nesse timeslot pode ter, no máximo, 1 reunião
        for c in range(C):
            solver.Add(sum([xDict[t][s][p][c] for s in xDict[t] for p in xDict[t][s] if c in xDict[t][s][p]]) <= 1)
            
        #Cada Colaborador tem de ter horário -> Obrigatório pela definição dos dicionários
        #Cada Colaborador apenas pode participar dos seus projetos -> Obrigatório pela definição dos dicionários
        
        #Cada Líder tem de participar do seu projeto OU SEJA
            #Cada Colaborador participa apenas de um projeto se o seu líder participar
        for s in xDict[t]:
            for p in xDict[t][s]:
                for c in xDict[t][s][p]:
                    solver.Add(xDict[t][s][p][c] <= xDict[t][s][p][projDict[p]['lider']])
                    
        #Cada Projeto apenas pode ser selecionado se houver 'quórum', o projeto só existe se o líder existe
            #Apenas se pode olhar para uma dada sala, ou podemos ver salas de outros projetos para algum colaborador
            #Número de participantes efetivos / total de colaboradores >= 1 / 2
            #Com cálculos chega-se a: 2*Número de participantes efetivos >= total de colaboradores
                #É necessário multiplicar pelo valor do líder para que não obriguemos a ter um somatório de 0's (pessoas não vão) >= quorum
        for p in range(P):
            for s in xDict[t]:
                if p in xDict[t][s]:
                    solver.Add(2*sum([xDict[t][s][p][c] for c in xDict[t][s][p]]) >= 
                               len(projDict[p]['colaboradores']) * xDict[t][s][p][projDict[p]['lider']])
        
    #Para cada Projeto, o número de vezes que foi selecionado tem de ser igual ou maior ao seu mínimo
    for p in range(P):
        solver.Add(sum([xDict[t][s][p][projDict[p]['lider']] for t in xDict for s in xDict[t] if p in xDict[t][s]]) >= projDict[p]['min'])
    
    #Número total de reuniões realizadas
    solver.Maximize(sum([xDict[t][s][p][projDict[p]['lider']] for t in xDict for s in xDict[t] for p in xDict[t][s]]))
    
    status = solver.Solve()
    if status == pywraplp.Solver.OPTIMAL:
        #Limitar os mínimos aos máximos descobertos pela solução anterior
        for p in range(P):
            solver.Add(sum([xDict[t][s][p][projDict[p]['lider']] for t in xDict for s in xDict[t] if p in xDict[t][s]]) >= 
                       sum([xDict[t][s][p][projDict[p]['lider']].solution_value() for t in xDict for s in xDict[t] if p in xDict[t][s]]))
    
        #Média das reuniões por participante (total de participações / total de participantes, que é constante)
        solver.Minimize(sum([xDict[t][s][p][c] for t in xDict for s in xDict[t] for p in xDict[t][s] for c in xDict[t][s][p]]))
    
        status = solver.Solve()
    
    if status == pywraplp.Solver.OPTIMAL:
        count = 0
        for t in xDict:
            print(f'Slot: {t}')
            for s in xDict[t]:
                for p in xDict[t][s]:
                    if projDict[p]['lider'] in xDict[t][s][p] and xDict[t][s][p][projDict[p]['lider']].solution_value() == 1:
                        l = projDict[p]['lider']
                        print(f'Projeto {p} de líder {l} na Sala {s}')
                        print("Colaboradores:")
                        for c in xDict[t][s][p]:
                            if xDict[t][s][p][c].solution_value() == 1:
                                print(f'{c}')
                                count += 1
                        print("")
            print("")
        print("Número médio de reuniões por colaborador: ")
        print(f"{count/C}")
    else:
        print("Nao é possivel construir o hórario")
        
scheduleStartUp(S, T, P, C, projetos, matrix)

AttributeError: type object 'Solver' has no attribute 'Optimal'